In [2]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

#Helpers
import data_loader as dl
import psf

## Preprocessing

### Load the pictures and extract the patches

In [2]:
directName = "Data/primitive/"

In [3]:
#Load the set of images and extract the patches
imgs    = dl.load_set(directName, 26)
patches = dl.extract_patch(len(imgs),320,imgs)

Loading 21 images


### Save patches in a new folder

In [4]:
for i in range(1,len(patches)):
    imageio.imwrite('Data/patches/patch'+str(i)+'.png',patches[i])

## Blur Detection

In [8]:
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout,UpSampling2D,concatenate
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation

Using TensorFlow backend.


In [11]:
from sklearn.model_selection import train_test_split

### Create the blured patches for training the model

In [8]:
directName = "sharpPatches/"

In [9]:
sharpPatches = dl.load_set(directName, 192)

Loading 192 images


In [10]:
for i in range(1,len(sharpPatches)):
    blurredPatch = psf.convolve(sharpPatches[i],psf.gauss_kern(11,3))
    imageio.imwrite('blurPatches11_3\patch'+str(i)+'.png',blurredPatch)

Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [6, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [1, 254]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [11,

KeyboardInterrupt: 

In [10]:
for i in range(1,len(sharpPatches)):
    blurredPatch = psf.convolve(sharpPatches[i],psf.gauss_kern(22,9))
    imageio.imwrite('blurPatches22_9\patch'+str(i)+'.png',blurredPatch)

### Load the training data

In [6]:
# slice the patches in order to have a window of 80x80
#Load data set and create X and Y 
X  = dl.extract_patch(192,80,dl.load_set("Data/sharpPatchesRaw/",192))
X1 = dl.extract_patch(191,80,dl.load_set("Data/blurPatches11_3/",192))
X2 = dl.extract_patch(191,80,dl.load_set("Data/blurPatches22_9/",192))
x  = np.concatenate((X,X1,X2))

Loading 192 images
Loading 191 images
Loading 191 images


In [9]:
#we need y to have shape (N,y) where N is the number of categories
y=to_categorical(np.concatenate((np.zeros(len(X)),np.ones(len(X1)),np.ones(len(X2))*2)))

### Create training and testing sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

### CNN Model

In [19]:
#create model to assess if a certain patch is blurred or not

model = Sequential()
#add model layers from paper : Assessing microscope image focus quality with deep learnin
window_size=80
model.add(Conv2D(32, kernel_size=5, activation='relu',padding='same', input_shape=(window_size,window_size,3), kernel_regularizer=l2(0.0005)))
model.add(MaxPooling2D((2, 2),strides=2))

model.add(Conv2D(64, kernel_size=5, activation='relu',padding='same',kernel_regularizer=l2(0.0005)))
model.add(MaxPooling2D((2, 2),strides=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))

#model.add(flatten(()))
model.add(Dense(3,activation='softmax'))



#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 80, 80, 32)        2432      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 40, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 20, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)             

### Train the model

In [16]:
from keras.callbacks import EarlyStopping

In [17]:
es = EarlyStopping(monitor='val_loss', mode='min')

In [20]:
model.fit(X_train, y_train, epochs=10,initial_epoch=0,validation_split=0.2,callbacks=[es])

Train on 4922 samples, validate on 1231 samples
Epoch 1/10
4922/4922 [==============================] - 135s 27ms/step - loss: 0.9433 - accuracy: 0.5177 - val_loss: 0.6050 - val_accuracy: 0.6279
Epoch 2/10
4922/4922 [==============================] - 125s 25ms/step - loss: 0.5386 - accuracy: 0.6924 - val_loss: 0.6996 - val_accuracy: 0.6052
